In [91]:
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt

np.random.seed(24)

%run ./DiffusionLunarKF.ipynb
%run ./CentralizedLunarKF.ipynb
%run ./FilterComparison.ipynb
%run ./VectorFormatting.ipynb

[0.29625406248427016, 0.2158641160259888, 0.30855146709925974, 0.06791110051253002, 0.4261028565513532, 0.6985049124279291, 0.5450395956052712, 0.3405131707471133]
[[0.11141925 0.29625406 0.21586412 0.30855147 0.0679111 ]
 [0.57389714 0.42610286 0.         0.         0.        ]
 [0.30149509 0.         0.69850491 0.         0.        ]
 [0.4549604  0.         0.         0.5450396  0.        ]
 [0.65948683 0.         0.         0.         0.34051317]]
Row Sums:  [1.0, 1.0, 1.0, 1.0, 1.0]


In [92]:
def MSE(vec):
    #C just is a reference for where we know there are edges in our network
    # adj = np.array([[1,1,1,1,1],[1,1,0,0,0],[1,0,1,0,0],[1,0,0,1,0],[1,0,0,0,1]])
    adj = np.array([[1,0,1],[0,1,0],[1,0,1]])

    C = convert_to_adj(vec,adj)
    n = len(C)

    D = 3*adj
    D_ckf = 3*np.ones((n,n))

    true_biases = np.array([[np.random.normal(0,np.sqrt(12/(c**2))) for _ in range(n)]]).T
    true_drifts = np.array([[np.random.normal(0,np.sqrt(0.1/(c**2))) for _ in range(n)]]).T
    # true_drifts = np.array([[0 for _ in range(n)]]).T

    F = np.array([[1,dt],[0,1]])
    F_full = np.kron(np.eye(n),F)

    x = c*np.vstack(tuple([np.array([true_biases[i],true_drifts[i]]) for i in range(n)]))

    # random initial estimates for each node

    x0 = [np.array([[np.random.normal(0,np.sqrt(12))],[np.random.normal(0,np.sqrt(0.1))]]) for i in range(n)]
    x0_cf = np.vstack(tuple(x0))
    # x0 = [np.array([[0],[0]]) for _ in range(n)]

    P = [100*np.copy(R(1)) for _ in range(n)]
    P_prev = np.block([[P[i] if i==j else np.zeros((2,2)) for j in range(n)] for i in range(n)])

    stations = [Station(i) for i in range(n)]

    filter_initialize(stations,D,x0,P)

    Q_10x10 = np.kron(np.eye(n),Q)

    kf = KalmanFilter(A = F_full, H = H_cf, Q = Q_10x10, R = R_cf, P = P_prev, x0 = x0_cf)

    iterations = 100

    # num_msmts = np.random.randint(0,10,(iterations,5))
    num_msmts = np.array([[3,3,3] for _ in range(iterations)])

    filter_outputs = run_both_filters(iterations, num_msmts,C,F_full,stations,kf, x)

    errors_df,errors_cf,P_hist_cf,P_hist_df,truth,measurements,predictions_cf,predictions_df = filter_outputs
    squared_error_iteration = lambda i: sum([x**2 for x in errors_df[i,:,0,0]])
    return sum([x**2 for x in [squared_error_iteration(i) for i in range(iterations)]])/(n*iterations)

In [93]:
print(MSE([0.5,0.5]))

[[0.5 0.  0.5]
 [0.  1.  0. ]
 [0.5 0.  0.5]]
124.62565867101549


In [95]:
x = sp.optimize.brute(MSE,(slice(0.1,0.9,0.1),slice(0.1,0.9,0.1)))

[[0.9 0.  0.1]
 [0.  1.  0. ]
 [0.9 0.  0.1]]
[[0.9 0.  0.1]
 [0.  1.  0. ]
 [0.8 0.  0.2]]
[[0.9 0.  0.1]
 [0.  1.  0. ]
 [0.7 0.  0.3]]
[[0.9 0.  0.1]
 [0.  1.  0. ]
 [0.6 0.  0.4]]
[[0.9 0.  0.1]
 [0.  1.  0. ]
 [0.5 0.  0.5]]
[[0.9 0.  0.1]
 [0.  1.  0. ]
 [0.4 0.  0.6]]
[[0.9 0.  0.1]
 [0.  1.  0. ]
 [0.3 0.  0.7]]
[[0.9 0.  0.1]
 [0.  1.  0. ]
 [0.2 0.  0.8]]
[[0.8 0.  0.2]
 [0.  1.  0. ]
 [0.9 0.  0.1]]
[[0.8 0.  0.2]
 [0.  1.  0. ]
 [0.8 0.  0.2]]
[[0.8 0.  0.2]
 [0.  1.  0. ]
 [0.7 0.  0.3]]
[[0.8 0.  0.2]
 [0.  1.  0. ]
 [0.6 0.  0.4]]
[[0.8 0.  0.2]
 [0.  1.  0. ]
 [0.5 0.  0.5]]
[[0.8 0.  0.2]
 [0.  1.  0. ]
 [0.4 0.  0.6]]
[[0.8 0.  0.2]
 [0.  1.  0. ]
 [0.3 0.  0.7]]
[[0.8 0.  0.2]
 [0.  1.  0. ]
 [0.2 0.  0.8]]
[[0.7 0.  0.3]
 [0.  1.  0. ]
 [0.9 0.  0.1]]
[[0.7 0.  0.3]
 [0.  1.  0. ]
 [0.8 0.  0.2]]
[[0.7 0.  0.3]
 [0.  1.  0. ]
 [0.7 0.  0.3]]
[[0.7 0.  0.3]
 [0.  1.  0. ]
 [0.6 0.  0.4]]
[[0.7 0.  0.3]
 [0.  1.  0. ]
 [0.5 0.  0.5]]
[[0.7 0.  0.3]
 [0.  1.  0. ]
 [0.

In [96]:
print(MSE(x))

[[0.79962891 0.         0.20037109]
 [0.         1.         0.        ]
 [0.17390625 0.         0.82609375]]
36.9947083193611
